In [1]:
from sleap_roots import DicotPipeline, YoungerMonocotPipeline, OlderMonocotPipeline, MultipleDicotPipeline, Series, find_all_h5_paths, find_all_slp_paths, load_series_from_h5s, load_series_from_slps
import pandas as pd
import numpy as np
from pathlib import Path

from sleap_roots.trait_pipelines import (
    DicotPipeline,
    MultipleDicotPipeline,
    YoungerMonocotPipeline,
    OlderMonocotPipeline,
    PrimaryRootPipeline
)

In [2]:
import base64
import requests
from IPython.display import Image, display

def mm_ink(graphbytes):
    """Given a bytes object holding a Mermaid-format graph, return a URL that will generate the image."""
    base64_bytes = base64.b64encode(graphbytes)
    base64_string = base64_bytes.decode("ascii")
    return "https://mermaid.ink/img/" + base64_string

def mm_display(graphbytes):
    """Given a bytes object holding a Mermaid-format graph, display it."""
    display(Image(url=mm_ink(graphbytes)))

def mm(graph):
    """Given a string containing a Mermaid-format graph, display it."""
    graphbytes = graph.encode("ascii")
    mm_display(graphbytes)

def mm_link(graph):
    """Given a string containing a Mermaid-format graph, return URL for display."""
    graphbytes = graph.encode("ascii")
    return mm_ink(graphbytes)

def mm_path(path):
    """Given a path to a file containing a Mermaid-format graph, display it"""
    with open(path, 'rb') as f:
        graphbytes = f.read()
    mm_display(graphbytes)

def save_mermaid_graph_image(graph, name):
    """Save the Mermaid graph image to a file."""
    graphbytes = graph.encode("ascii")
    url = mm_ink(graphbytes)
    response = requests.get(url)
    if response.status_code == 200:
        image_file = f"{name}_Mermaid_Graph.png"
        with open(image_file, 'wb') as file:
            file.write(response.content)
        print(f"{name} Mermaid graph image saved to {image_file}")
    else:
        print("Failed to fetch the Mermaid graph image.")

def generate_mermaid_graph_and_write_to_file(pipeline, name):
    trait_definitions = pipeline().define_traits()
    graph = "graph LR\n"
    unique_traits = set()

    for trait_def in trait_definitions:
        for input_trait in trait_def.input_traits:
            graph += f"    {input_trait} --> {trait_def.name}\n"
            unique_traits.add(input_trait)
            unique_traits.add(trait_def.name)
    
    # Write to a markdown file
    markdown_graph = f"```mermaid\n{graph}```\n"
    file_name = f"{name}_Mermaid_Graph.md"
    with open(file_name, 'w') as file:
        file.write(markdown_graph)
        file.write(f"\nDistinct Traits for {name}: {len(unique_traits)}\n")
    
    print(f"{name} Mermaid graph written to {file_name}")
    
    # Display in Jupyter Notebook
    try:
        mm(graph)
    except ImportError:
        print("IPython is not available. The graph cannot be displayed in the notebook.")
    
    # Save the image
    save_mermaid_graph_image(graph, name)
    
    return markdown_graph, len(unique_traits)

# Example usage
# Assuming 'MyPipeline' is your pipeline class
# graph, num_traits = generate_mermaid_graph_and_write_to_file(MyPipeline, "MyPipeline")


In [3]:
younger_monocot_pipeline = YoungerMonocotPipeline() # Requires crown and primary root predictions
older_monocot_pipeline = OlderMonocotPipeline() # Requires crown roots predictions
dicot_pipeline = DicotPipeline() # Requires primary and lateral root predictions
multiple_dicot_pipeline = MultipleDicotPipeline() # Requires primary and lateral root predictions, and CSV with expected plant count per cylinder
primary_root_pipeline = PrimaryRootPipeline()

In [4]:
pipelines = {
    #"DicotPipeline": DicotPipeline,
    #"YoungerMonocotPipeline": YoungerMonocotPipeline,
    #"OlderMonocotPipeline": OlderMonocotPipeline,
    #"MultipleDicotPipeline": MultipleDicotPipeline,
    "PrimaryRootPipeline": PrimaryRootPipeline
}

for name, pipeline in pipelines.items():
    graph, distinct_traits = generate_mermaid_graph_and_write_to_file(pipeline, name)
    print(f"Mermaid Graph for {name}:")
    print(graph)
    print(f"Distinct Traits for {name}: {distinct_traits}\n")

PrimaryRootPipeline Mermaid graph written to PrimaryRootPipeline_Mermaid_Graph.md


PrimaryRootPipeline Mermaid graph image saved to PrimaryRootPipeline_Mermaid_Graph.png
Mermaid Graph for PrimaryRootPipeline:
```mermaid
graph LR
    primary_pts --> primary_max_length_pts
    primary_max_length_pts --> primary_proximal_node_ind
    primary_max_length_pts --> primary_distal_node_ind
    primary_max_length_pts --> primary_angle_proximal
    primary_proximal_node_ind --> primary_angle_proximal
    primary_max_length_pts --> primary_angle_distal
    primary_distal_node_ind --> primary_angle_distal
    primary_max_length_pts --> primary_length
    primary_max_length_pts --> primary_base_pt
    primary_max_length_pts --> primary_tip_pt
    primary_base_pt --> primary_base_pt_x
    primary_base_pt --> primary_base_pt_y
    primary_tip_pt --> primary_tip_pt_x
    primary_tip_pt --> primary_tip_pt_y
    primary_base_pt --> primary_base_tip_dist
    primary_tip_pt --> primary_base_tip_dist
    primary_length --> curve_index
    primary_base_tip_dist --> curve_index
    primary_